In [1]:
#2021 05 08 Геолокация круизных теплоходов. Скрипт формирования карты и отправки по эл.почте по расписанию заданному в Планировщике.

In [2]:
#работаю в директории:
import os

os.chdir('C:\\Python\\Geo')
os.getcwd()

'C:\\Python\\Geo'

In [4]:
# %pip install pymysql
# %pip install folium

In [3]:
import numpy as np
import pandas as pd
import datetime
import openpyxl
import xlsxwriter

import pymysql
import pymysql.cursors

In [6]:
import folium
from folium.plugins import MarkerCluster

In [7]:
import win32com.client as win32

import seaborn as sns
import matplotlib.pyplot as plt

from copy import deepcopy

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [8]:
# подключение к базе даных координат GPS


connection = pymysql.connect(host='XX.XXX.XX.XX',
                             user='XX',
                             password='XXX',                             
                             db='XXXX', 
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
 
connection.open
connection.cursor()
cur = connection.cursor()


sql = "SELECT * FROM ships"

cur.execute(sql) 

desc = cur.description
row = cur.fetchall()
df = pd.DataFrame(row)

In [9]:
#поле с датой
df["date"] = pd.to_datetime(df["request_time"])

In [10]:
#добавляю год и месяц и тыс.₽ в отдельные столбцы
df["Year"] = df["date"].dt.year
df["Month"] = df["date"].dt.month
df["Day"] = df["date"].dt.day

In [13]:
# df.tail()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175642 entries, 0 to 175641
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype                                
---  ------         --------------   -----                                
 0   name           175642 non-null  object                               
 1   date_and_time  175642 non-null  object                               
 2   latitude       175642 non-null  object                               
 3   longitude      175642 non-null  object                               
 4   speed          175642 non-null  object                               
 5   true_bearing   175642 non-null  object                               
 6   valid          175642 non-null  object                               
 7   satellites     175642 non-null  object                               
 8   request_time   175642 non-null  object                               
 9   date           175642 non-null  datetime64[ns, pytz.FixedOf

In [14]:
df['name'].unique()

array(['Bulgakov', 'Rublev', 'Krilov', 'Viktoriya', 'Krasin', 'Grin', '',
       'Karamzin', 'Obrazcov'], dtype=object)

In [15]:
#т/х Рублёв
df_Grin = df[(df['name']=='Grin') & (df['valid']=='true')]

df_Grin.tail()

,name,date_and_time,latitude,longitude,speed,true_bearing,valid,satellites,request_time,date,Year,Month,Day
175614,Grin,sep/22/2001 14:09:44,55.774833333333,49.090483333333,0.000000 km/h,173.399994 deg. True,true,12,2021-05-08T17:09:51+03:00,2021-05-08 17:09:51+03:00,2021,5,8
175620,Grin,sep/22/2001 14:14:09,55.774838333333,49.090516666667,0.000000 km/h,173.399994 deg. True,true,12,2021-05-08T17:14:53+03:00,2021-05-08 17:14:53+03:00,2021,5,8
175626,Grin,sep/22/2001 14:19:27,55.774838333333,49.090533333333,0.000000 km/h,173.399994 deg. True,true,10,2021-05-08T17:19:51+03:00,2021-05-08 17:19:51+03:00,2021,5,8
175632,Grin,sep/22/2001 14:24:45,55.774835,49.09055,0.000000 km/h,173.399994 deg. True,true,12,2021-05-08T17:24:51+03:00,2021-05-08 17:24:51+03:00,2021,5,8
175638,Grin,sep/22/2001 14:29:10,55.774845,49.09026,0.000000 km/h,173.399994 deg. True,true,10,2021-05-08T17:29:50+03:00,2021-05-08 17:29:50+03:00,2021,5,8


In [16]:
# беру только последние 50 точек т/х Грин
df_Grin_50 = df_Grin.tail(50)

df_Grin_50.tail(15)

,name,date_and_time,latitude,longitude,speed,true_bearing,valid,satellites,request_time,date,Year,Month,Day
175554,Grin,sep/22/2001 13:19:23,55.774828333333,49.090306666667,0.000000 km/h,173.399994 deg. True,true,9,2021-05-08T16:19:50+03:00,2021-05-08 16:19:50+03:00,2021,5,8
175560,Grin,sep/22/2001 13:24:41,55.774848333333,49.090308333333,0.000000 km/h,173.399994 deg. True,true,10,2021-05-08T16:24:51+03:00,2021-05-08 16:24:51+03:00,2021,5,8
175566,Grin,sep/22/2001 13:29:06,55.774845,49.090315,0.000000 km/h,173.399994 deg. True,true,10,2021-05-08T16:29:51+03:00,2021-05-08 16:29:51+03:00,2021,5,8
175572,Grin,sep/22/2001 13:34:24,55.774841666667,49.090316666667,0.000000 km/h,173.399994 deg. True,true,11,2021-05-08T16:34:51+03:00,2021-05-08 16:34:51+03:00,2021,5,8
175578,Grin,sep/22/2001 13:39:42,55.774838333333,49.090308333333,0.000000 km/h,173.399994 deg. True,true,12,2021-05-08T16:39:51+03:00,2021-05-08 16:39:51+03:00,2021,5,8
175584,Grin,sep/22/2001 13:44:07,55.77484,49.09055,0.000000 km/h,173.399994 deg. True,true,11,2021-05-08T16:44:50+03:00,2021-05-08 16:44:50+03:00,2021,5,8
175590,Grin,sep/22/2001 13:49:25,55.77484,49.090483333333,0.000000 km/h,173.399994 deg. True,true,12,2021-05-08T16:49:51+03:00,2021-05-08 16:49:51+03:00,2021,5,8
175596,Grin,sep/22/2001 13:54:43,55.77484,49.090483333333,0.000000 km/h,173.399994 deg. True,true,12,2021-05-08T16:54:51+03:00,2021-05-08 16:54:51+03:00,2021,5,8
175602,Grin,sep/22/2001 13:59:08,55.774838333333,49.090433333333,0.000000 km/h,173.399994 deg. True,true,11,2021-05-08T16:59:50+03:00,2021-05-08 16:59:50+03:00,2021,5,8
175608,Grin,sep/22/2001 14:04:26,55.774835,49.0905,0.000000 km/h,173.399994 deg. True,true,12,2021-05-08T17:04:51+03:00,2021-05-08 17:04:51+03:00,2021,5,8


In [18]:
#карта с крайними 50 координатами т/х Грин

map_Grin_50 = folium.Map(location=[df_Grin_50['latitude'][-1:], df_Grin_50['longitude'][-1:]], zoom_start=11)
marker_cluster = MarkerCluster().add_to(map_Grin_50)

for index, row in df_Grin_50.iterrows():
    folium.Marker(location=[row["latitude"], row["longitude"]], popup=row[["name", "date"]], color='blue').add_to(marker_cluster) 

map_Grin_50.add_child(folium.LatLngPopup())  
map_Grin_50

In [20]:
#только последние верные координаты
gr = df[df['valid']=='true'].groupby('name', as_index=False).tail(1)

In [21]:
#карта со всеми судами. Нужно только найти среднюю точку чтобы автоматически карта ставала по центру.

map_gr = folium.Map(location=[gr['latitude'][-1:], gr['longitude'][-1:]], tiles='Stamen Terrain', zoom_start=5) # 'Stamen Terrain' 'Stamen Toner'

marker_cluster = MarkerCluster().add_to(map_gr)

for index, row in gr.iterrows():
    folium.Marker(location=[row["latitude"], row["longitude"]], popup=row[["name", "date"]], color='blue').add_to(marker_cluster) 

map_gr.add_child(folium.LatLngPopup())  

map_gr

In [23]:
map_Grin_50.save('Grin.html')
map_gr.save('map_gr.html')

In [24]:
#пакеты для работы с почтой

import smtplib
import mimetypes
import email
import email.mime.application
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [27]:
# Отправляю по почте несколько файлов

msg = MIMEMultipart()
msg['Subject'] = 'МТФ. Геолокация. Навигация 2021'
msg['From'] = 'report_sender@mosrp.ru'
msg['To'] = 'МТФ'

body = MIMEText('Здравствуйте, <br>                 Отчет по геолокации т/х <br>                 Отчет сформирован на основании базы накопления GPS координат т/х.<br> <br>                 <font color="blue"> С уважением,<br> Смоленышев Олег <br>                 smolenyshev@mosrp.ru <br>                 +7 916 014 37 70 ', 'html')

msg.attach(body)


dir_path = 'C:\\Python\\Geo'
files = ["map_gr.html", 'Grin.html']

for f in files:  # add files to the message
        file_path = os.path.join(dir_path, f)
        att = MIMEApplication(open(file_path, "rb").read(), _subtype="html")
        att.add_header('Content-Disposition','attachment', filename=f)
        msg.attach(att)


s = smtplib.SMTP_SSL('XX.XXXX.XX', 465)
s.login('XXXX', 'XXXXX')

a = ['smolenyshev@mosrp.ru']

for i in range(0, len(a)):
    s.sendmail('report_sender@mosrp.ru', a[i], msg.as_string())

print("Отправлены файлы html с картами и расположением теплоходов")

s.quit()

Отправлены файлы html с картами и расположением теплоходов


(221, b'2.0.0 SMTP closing connection')